In [10]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

train = pd.read_csv('train_data.csv', index_col = 0).reset_index(drop=True)
plus = pd.read_csv('0829_300_3.csv')
plus1 = pd.read_csv('0831_300_1.csv')
plus2 = pd.read_csv('0901_300_1.csv')

plus1.drop('category', axis = 1, inplace = True)
plus2.drop('category', axis = 1, inplace = True)
plus1.drop('신문사', axis = 1, inplace = True)
plus2.drop('신문사', axis = 1, inplace = True)

okt = Okt()

train.title = train.title.str.replace('[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ一-龥]', ' ')
plus.title = plus.title.str.replace('[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ一-龥]', ' ')
plus1.title = plus1.title.str.replace('[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ一-龥]', ' ')
plus2.title = plus2.title.str.replace('[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ一-龥]', ' ')


crawed = pd.concat([plus, plus1, plus2]).drop_duplicates(subset = ['title'], keep = 'first').reset_index(drop=True)

train.drop_duplicates(inplace = True)
train.reset_index(drop= True, inplace = True)

train_x, valid_x,_, _ = train_test_split(train, train['topic_idx'],
                                         test_size = 0.3,
                                         random_state = 10)


total = pd.concat([train_x, crawed]).reset_index(drop = True)
valid_x.reset_index(drop=True, inplace = True)


f = pd.DataFrame()
for i in range(len(total.title)):
    c = okt.pos(total.title[i])
    d = [y for (y, x) in c if (x=='Noun')|(x == 'Alpha')|(x=='Foreign')]

    e = ''
    for j in range(len(d)):
        if j < (len(d) - 1):
            e += d[j]+' '
        else:
            e += d[j]
    f = pd.concat([f, pd.DataFrame(np.array([e, total.topic_idx[i]]).reshape(1,2))])
    
    

g = pd.DataFrame()
for i in range(len(valid_x.title)):
    c = okt.pos(valid_x.title[i])
    d = [y for (y, x) in c if (x=='Noun')|(x == 'Alpha')|(x=='Foreign')]

    e = ''
    for j in range(len(d)):
        if j < (len(d) - 1):
            e += d[j]+' '
        else:
            e += d[j]
    g = pd.concat([g, pd.DataFrame(np.array([e, valid_x.topic_idx[i]]).reshape(1,2))])
    


f.columns = ['title', 'topic_idx']
f.reset_index(drop=True, inplace = True)


g.columns = ['title', 'topic_idx']
g.reset_index(drop=True, inplace = True)

f.topic_idx = f.topic_idx.astype('int8')
g.topic_idx = g.topic_idx.astype('int8')

data = f['title']

train_sentences = f['title']
valid_sentences = g['title']

train_labels = f['topic_idx']
valid_labels = g['topic_idx']

In [11]:
f.loc[f.duplicated()]

,title,topic_idx
4782,평양 정상회담 대화 남북 정상,6
5770,국내 주식 펀드 나흘 자금 유출,1
5875,평양 정상회담 평양 시민 인사 문 대통령,6
8331,국내 주식 펀드 거래 일 자금 유출,1
8924,도자기 관람객,3
...,...,...
52830,아프간 앵커 그 뒤 총 명의 탈레반,4
52845,아프간 통역사 가족 개 고양이 전세기 탈출 영국인,4
52876,탈레반 카불 공항 장악 이제 탈출 루트 파키스탄 국경 르포,4
52959,삼성 엔비디아 ARM 인수 반대 아마존 테슬라 우려,4


In [12]:
g.loc[g.duplicated()]

,title,topic_idx
7748,전역 꽃시계 봄옷,3
9845,그래픽 코스피 코스닥 추이,1
10947,두바이 유 가격 반등 배럴 달러,1
11283,류현진 이닝 실점 홈런 루타 첫 타점 승 눈앞,5
11910,파워 직스 분기 영업 익 억원 감소,1
12058,NBA 휴스턴 쿼터 점 플레이오프 R 진출 승,5
12364,거창 국제 연극제 강탈,3


In [13]:
# f; train set / g; valid set

f.drop_duplicates().reset_index(drop=True).to_csv('f.csv', index = None)
g.drop_duplicates().reset_index(drop=True).to_csv('g.csv', index = None)

In [18]:
f = pd.DataFrame()
train_x.reset_index(drop=True, inplace=True)
for i in range(len(train_x.title)):
    c = okt.pos(train_x.title[i])
    d = [y for (y, x) in c if (x=='Noun')|(x == 'Alpha')|(x=='Foreign')]

    e = ''
    for j in range(len(d)):
        if j < (len(d) - 1):
            e += d[j]+' '
        else:
            e += d[j]
    f = pd.concat([f, pd.DataFrame(np.array([e, train_x.topic_idx[i]]).reshape(1,2))])
    
    

g = pd.DataFrame()
for i in range(len(valid_x.title)):
    c = okt.pos(valid_x.title[i])
    d = [y for (y, x) in c if (x=='Noun')|(x == 'Alpha')|(x=='Foreign')]

    e = ''
    for j in range(len(d)):
        if j < (len(d) - 1):
            e += d[j]+' '
        else:
            e += d[j]
    g = pd.concat([g, pd.DataFrame(np.array([e, valid_x.topic_idx[i]]).reshape(1,2))])
    


f.columns = ['title', 'topic_idx']
f.reset_index(drop=True, inplace = True)


g.columns = ['title', 'topic_idx']
g.reset_index(drop=True, inplace = True)

f.topic_idx = f.topic_idx.astype('int8')
g.topic_idx = g.topic_idx.astype('int8')

data = f['title']

train_sentences = f['title']
valid_sentences = g['title']

train_labels = f['topic_idx']
valid_labels = g['topic_idx']

# f; train set / g; valid set

f.drop_duplicates().reset_index(drop=True).to_csv('f1.csv', index = None)
g.drop_duplicates().reset_index(drop=True).to_csv('g1.csv', index = None)